In [1]:
import os
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier


In [2]:
train_data_files = ['cross', 'dead', 'left', 'right', 'straight', 'threeway']
# NUM_CLASSES = 6
NUM_CLASSES = len(train_data_files)

epochs = 120
batch_size = 100

RATIO_DROP_OUT1 = 0.25
RATIO_DROP_OUT2 = 0.50



In [3]:
def split_data():
    files = os.listdir('./data')
    X = []
    Y = []
    all_data_set = []
    labels = []


    for i in range(NUM_CLASSES):
        try:
            # header = 列名
            file_name = "./data/" + files[i]
            data_set = pd.read_csv(file_name, header=None)
            all_data_set.append(data_set)
        
            num_data_set = len(data_set)
            print(num_data_set)

            # one_hot_vectorを作りラベルとして追加
            tmp = np.zeros((num_data_set, NUM_CLASSES))
            tmp[:, i] = 1
            labels.append(tmp)
        except pd.io.common.EmptyDataError:
            print("ERROR: {} is empty".format(file_name))

    X = pd.concat(all_data_set)
    X = X.fillna(0)
    Y = np.concatenate(labels, axis=0)

#    _, DIM_input_data = data_set.shape

    X_train, X_validation_and_test, Y_train, Y_validation_and_test = train_test_split(X, Y,train_size=0.6, test_size=0.4)
    X_validation, X_test, Y_validation, Y_test = train_test_split(X_validation_and_test, Y_validation_and_test, train_size=0.5, test_size=0.5)

    return X_train, X_validation, X_test, Y_train, Y_validation, Y_test

In [4]:
def plt_result(epochs, history):
    plt.plot(range(1, epochs+1), history.history['acc'], label="training")
    plt.plot(range(1, epochs+1), history.history['val_acc'], label="validation")
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [5]:
def param():
    ret = {
        'out_dim':[100, 200],
#        'optimizer':["adam", "adagrad"]
    }

    return ret

In [6]:
def make_model(activation="relu", optimizer="adam", out_dim=100):
    model = Sequential()
    model.add(Dense(out_dim, input_dim=726, activation=activation))
    model.add(Dropout(RATIO_DROP_OUT1))

    model.add(Dense(out_dim, activation=activation))
    model.add(Dropout(RATIO_DROP_OUT2))

    model.add(Dense(NUM_CLASSES, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [7]:
X_train, X_validation, X_test, Y_train, Y_validation, Y_test = split_data()

# Retrieve model and parameter into GridSearchCV
model = KerasClassifier(build_fn=make_model, verbose=0)

param_grid = param()
grid = GridSearchCV(estimator=model, param_grid=param_grid)

# Run grid search
grid_result = grid.fit(X_train, Y_train)

# Get the best score and the optimized mode
print (grid_result.best_score_)
print (grid_result.best_params_)

# Now see the optimized model
#model = make_model(out_dim=grid_result.best_params_['out_dim'])
#model.summary()

164
285
112
ERROR: ./data/right.csv is empty
3798
696
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Instructions for updating:
Use tf.cast instead.
0.7606330369708171
{'out_dim': 100}


In [8]:
plt.plot(range(1, epochs+1), history.history['acc'], label="training")
plt.plot(range(1, epochs+1), history.history['val_acc'], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined